In [1]:
import numpy as np
import databricks.koalas as ks
from pyspark.sql import SparkSession
import os
#import sys
from scipy.sparse import csr_matrix
import geopandas as gpd
import pickle
from pathlib import Path
import argparse
from pyspark.sql.functions import expr, to_timestamp, col, unix_timestamp, first, sum, count
from pyspark.sql.types import *
from pyspark.sql.types import LongType, IntegerType

In [2]:
# Datapath file CSV (dataset)
pathCSV = 'data/trips2323_new.csv'

# Path ShapeFile
pathShapeF = 'data/shapefile/shapefile_2323.shp'

# Path for results
pathRes = 'results/'

# Skip division per 0
np.seterr(divide='ignore', invalid='ignore')



def preprocessing(df):
    #df_cache = df.spark.cache()
    df = df.rename(columns={'from_zone':'from_zone_fid', 'from_timedate':'from_timedate_gmt', 'to_zone':'to_zone_fid', 'to_timedate':'to_timedate_gmt'})

    df = df.dropna(subset=['to_zone_fid'])
    fill = {'from_zone_fid': -1}
    df = df.fillna(value=fill)


    # Initial Filter
    # (df.triptime_s < 3278)
    df = df.loc[ (df.tripdistance_m > 0) & (df.stoptime_s > 0)]

    #df = df.sort_values(by=['from_timedate_gmt', 'to_timedate_gmt'])
    #df = df.reset_index(drop = True)

    df_spark = df.to_spark()

    df_spark = df_spark.withColumn("from_timedate_gmt", expr("substring(from_timedate_gmt, 1, length(from_timedate_gmt)-3)"))
    df_spark = df_spark.withColumn("to_timedate_gmt", expr("substring(to_timedate_gmt, 1, length(to_timedate_gmt)-3)"))
    #df_spark = df_spark.withColumn("from_timedate_gmt", df_spark.from_timedate_gmt.substr(1, 6))

    df = df_spark.to_koalas()

    #df_cache.spark.unpersist()

    return df
# second to hour
def square(s):
    return s/3600


In [3]:
spark = SparkSession.builder\
        .master("local[1]")\
        .appName("Pyspark")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()
spark.catalog.clearCache()
    
    
# Upload CSV datas
df = ks.read_csv(pathCSV, sep=';', dtype = {'from_timedate':str, 'to_timedate':str})
#df = df.spark.cache()
df = df.spark.repartition(30)
df = preprocessing(df)
#   print(df.head())
#   exit()
print('-----------OUTPUT------------')
#df_cache = df.spark.cache()
df = df.rename(columns={'from_zone':'from_zone_fid', 'from_timedate':'from_timedate_gmt', 'to_zone':'to_zone_fid', 'to_timedate':'to_timedate_gmt'})
df = df.dropna(subset=['to_zone_fid'])
fill = {'from_zone_fid': -1}
df = df.fillna(value=fill)
# Iitial Filter
# (f.triptime_s < 3278)
df = df.loc[ (df.tripdistance_m > 0) & (df.stoptime_s > 0)]# & (df.to_zone_fid != -1)]
#print(df.stoptime_s.max())
df.stoptime_s = df.stoptime_s.apply(square)
#test = df.stoptime_s.to_numpy()

#ax = 
#df.stoptime_s.plot.kde(bw_method=3)  

22/06/14 18:08:20 WARN Utils: Your hostname, gpu2-Standard-PC-Q35-ICH9-2009 resolves to a loopback address: 127.0.1.1; using 10.0.0.205 instead (on interface enp6s18)
22/06/14 18:08:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/14 18:08:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/06/14 18:08:22 WARN Utils: Service 'SparkUI' could not bind on port 4050. Attempting port 4051.


-----------OUTPUT------------


22/06/14 18:09:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/14 18:09:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/14 18:10:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
/home/gpu2/.local/lib/python3.8/site-packages/pyspark/sql/pandas/functions.py:389: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


In [ ]:
df.loc[ df.stoptime_s < 0]

22/06/14 16:25:15 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/14 16:25:15 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/14 16:26:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [4]:
def stoptime_bound(df):
    #Upper Bound e Lower Bound
    print(df.stoptime_s.describe())
    Q1 = df.stoptime_s.describe().to_list()[4]
    Q2 = df.stoptime_s.describe().to_list()[5]
    Q3 = df.stoptime_s.describe().to_list()[6]
    IQR = Q3 - Q1
    L = Q1 - (1.5 * IQR)
    U = Q3 + (1.5 * IQR)
    print('Lower: ', L)
    print('Upper', U)

In [5]:
stoptime_bound(df)

22/06/14 18:10:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/14 18:11:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/14 18:11:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


count    1.336024e+07
mean     5.320176e+00
std      2.853858e+01
min      2.777778e-04
25%      1.402778e-01
50%      7.661111e-01
75%      4.816667e+00
max      8.728416e+03
Name: stoptime_s, dtype: float64


22/06/14 18:11:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/14 18:12:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/14 18:12:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/14 18:13:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/14 18:14:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/14 18:14:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/14 1

Lower:  -6.874305555555556
Upper 11.83125


In [6]:
len(df)

13360243

In [9]:
len(df.loc[ df.stoptime_s < 12])

22/06/07 10:47:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/07 10:48:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/07 10:48:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


11751142

In [10]:
13340482 - 11751142

1589340

In [11]:
1589340/13340482

0.11913662489856064

In [ ]:
df.stoptime_s.plot.kde(bw_method=0.001)  